## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
users_data = pd.read_csv('/datasets/users_behavior.csv')
print(users_data.info())
users_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Предобработка данных не требуется. Посмотрим, сколько пользователей на каждом тарифе.

In [3]:
users_data_pt = users_data.pivot_table(index='is_ultra', values='calls', aggfunc='count')
users_data_pt

,calls
is_ultra,
0,2229
1,985


На тарифе "Смарт" пользователей более чем в 2 раза больше, чем на тарифе "Ультра". Это означает, что при построении моделей необходимо использовать гиперпараметр class_weight со значением balanced.

## 2. Разбейте данные на выборки

In [4]:
#делим данные на обучающую выборку и остаток
df_train, df_rest = train_test_split(users_data, test_size=0.25, random_state=12345)
df_train_features = df_train.drop('is_ultra', axis=1)
df_train_target = df_train['is_ultra']

#теперь остаток делим на валидационную и тестовую выборки
df_valid, df_test = train_test_split(df_rest, test_size=0.25, random_state=12345)
df_valid_features = df_valid.drop('is_ultra', axis=1)
df_valid_target = df_valid['is_ultra']
df_test_features = df_test.drop('is_ultra', axis=1)
df_test_target = df_test['is_ultra']

## 3. Исследуйте модели

Задача - построить модель, которая будет предсказывать тариф. Это задача классификации. Лучшую модель будем выбирать из решающего дерева (DecisionTreeClassifier), случайного леса (RandomForestClassifier) и логистической регрессии (LogisticRegression). Для каждой модели будем перебирать гиперпараметры и рассчитывать качество модели. Использовать будем следующие гиперпараметры:

<u>Решающее дерево.</u>
- max_depth - перебираем от 1 (пень) до 4 (т.к. максимум 4 признака)
- min_samples_split = 2
- min_samples_leaf = 1

<u> Случайный лес.</u>
- n_estimators - перебираем от 10 до 50 с шагом 10

In [5]:
#решающее дерево
for i in range(1, 5):
    tree_model = DecisionTreeClassifier(max_depth=i,
                                        random_state=12345,
                                        min_samples_split=2,
                                        min_samples_leaf=1,
                                       class_weight='balanced')
    tree_model.fit(df_train_features, df_train_target) #обучаем модель на обучающей выборке
    tree_model_predictions = tree_model.predict(df_valid_features) #предсказываем значения на валидационной выборке
    accuracy = accuracy_score(df_valid_target, tree_model_predictions) #считаем точность модели
    print('Решающее дерево, глубина={}, точность='.format(i), accuracy)

Решающее дерево, глубина=1, точность= 0.7545605306799337
Решающее дерево, глубина=2, точность= 0.7877280265339967
Решающее дерево, глубина=3, точность= 0.7943615257048093
Решающее дерево, глубина=4, точность= 0.746268656716418


In [6]:
#случайный лес
for i in range(10, 51, 10):
    forest_model = RandomForestClassifier(n_estimators=i, random_state=12345, class_weight='balanced')
    forest_model.fit(df_train_features, df_train_target) #обучаем модель
    forest_model_predictions = forest_model.predict(df_valid_features) #предсказываем значения на валидационной выборке
    accuracy = accuracy_score(df_valid_target, forest_model_predictions) #считаем точность
    print('Случайный лес, N деревьев={}, точность='.format(i), accuracy)

Случайный лес, N деревьев=10, точность= 0.7943615257048093
Случайный лес, N деревьев=20, точность= 0.7976782752902156
Случайный лес, N деревьев=30, точность= 0.8009950248756219
Случайный лес, N деревьев=40, точность= 0.7943615257048093
Случайный лес, N деревьев=50, точность= 0.7927031509121062


In [7]:
#логистическая регрессия
regression_model = LogisticRegression(random_state=12345, class_weight='balanced')
regression_model.fit(df_train_features, df_train_target) #обучаем модель
regression_model_predictions = regression_model.predict(df_valid_features) #предсказываем значения
accuracy = accuracy_score(df_valid_target, regression_model_predictions) #считаем точность
print('Логистическая регрессия, точность=', accuracy)

Логистическая регрессия, точность= 0.6434494195688225


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Вывод:** наилучшие результаты дает случайный лес. Наибольшая точность достигается при N деревьев = 30. Точность в данном случае = 0.8, при этом расчёт идёт достаточно быстро. При большем количестве деревьев расчёт идёт дольше, а точность уменьшается, т.е. возникает переобучение модели. К точности 0.8 приближается решающее дерево с гиперпараметром max_depth=3. Наименьшая точность у логистической регрессии (0.64).

В дальнейшем будем использовать случайный лес с N деревьев = 30 как модель с наивысшей точностью на валидационной выборке. Осталось проверить эту модель на тестовой выборке.

## 4. Проверьте модель на тестовой выборке

In [8]:
model = RandomForestClassifier(n_estimators=30, random_state=12345, class_weight='balanced') #задаем параметры модели
model.fit(df_train_features, df_train_target) #обучаем модель на обучающей выборке
model_predictions = model.predict(df_test_features) #предсказываем значения на тестовой выборке
accuracy = accuracy_score(df_test_target, model_predictions) #считаем качество модели на тестовой выборке
print('Случайный лес, N деревьев = 50, точность =',accuracy)

Случайный лес, N деревьев = 50, точность = 0.7761194029850746


Итого получаем точность 0.8 на валидационной выборке и 0.78 на тестовой выборке. В рамках данного проекта такая точность считается допустимой.

## 5. (бонус) Проверьте модели на адекватность

Ранее было посчитано, что большинство пользователей пользуется тарифом "Смарт". То есть если модель будет предсказывать только 0, точность будет более 50%. Посчитаем точность такой модели: accuracy = (количество правильных ответов) / (количество объектов)

In [9]:
accuracy = 2229 / (2229 + 985)
print(accuracy)

0.693528313627878


Точность составила 0.69. Это выше, чем точность логистической регрессии, которая составила 0.64, что говорит о том, что в рамках поставленной задачи пользоваться логистической регрессией нельзя. Точность других моделей (решающего дерева и случайного леса) выше. Эти модели пригодны для анализа и рекомендаций тарифов.